In [1]:
!pip install pyspark
!pip install delta-spark

import pyspark
from delta import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=6d11ed4515313879f2da3568caec43e095ce5ba23b023a83d7f86ab4693a15bb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/madrid-airbnb-data/calendar.csv
/kaggle/input/madrid-airbnb-data/listings.csv
/kaggle/input/madrid-airbnb-data/listings_detailed.csv
/kaggle/input/madrid-airbnb-data/neighbourhoods.geojson
/kaggle/input/madrid-airbnb-data/reviews.csv
/kaggle/input/madrid-airbnb-data/reviews_detailed.csv
/kaggle/input/madrid-airbnb-data/neighbourhoods.csv
/kaggle/input/barcelona-monthly-subway-users-2020/Barcelona_Monthly_Subway_Users.xlsx


In [3]:
airbnb_df = pd.read_csv('/kaggle/input/madrid-airbnb-data/calendar.csv')

print(airbnb_df) 

         listing_id        date available   price adjusted_price  \
0            167183  2021-04-15         f  $45.00         $45.00   
1              6369  2021-04-15         t  $60.00         $60.00   
2              6369  2021-04-16         t  $60.00         $60.00   
3              6369  2021-04-17         t  $60.00         $60.00   
4              6369  2021-04-18         t  $60.00         $60.00   
...             ...         ...       ...     ...            ...   
7161251    49103093  2022-04-10         f  $28.00         $35.00   
7161252    49103093  2022-04-11         f  $28.00         $35.00   
7161253    49103093  2022-04-12         f  $28.00         $35.00   
7161254    49103093  2022-04-13         f  $28.00         $35.00   
7161255    49103093  2022-04-14         f  $28.00         $35.00   

         minimum_nights  maximum_nights  
0                   1.0             5.0  
1                   1.0          1125.0  
2                   1.0          1125.0  
3              

In [4]:
airbnb_df['Month'] = pd.to_datetime(airbnb_df['date']).dt.strftime('%b')

In [5]:
subway_df = pd.read_excel('/kaggle/input/barcelona-monthly-subway-users-2020/Barcelona_Monthly_Subway_Users.xlsx', 
                          sheet_name=1, 
                          skiprows=4,
                          usecols="B:C",
                          names=['Month', 'Subway_Passengers'])

subway_df

,Month,Subway_Passengers
0,Jan,35220
1,Feb,35288
2,Mar,17463
3,Apr,3247
4,May,6800
5,Jun,13633
6,Jul,17508
7,Aug,12994
8,Sep,18374
9,Oct,20199


In [6]:
df = pd.merge(airbnb_df, subway_df, on='Month', how='left')
df

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,Month,Subway_Passengers
0,167183,2021-04-15,f,$45.00,$45.00,1.0,5.0,Apr,3247
1,6369,2021-04-15,t,$60.00,$60.00,1.0,1125.0,Apr,3247
2,6369,2021-04-16,t,$60.00,$60.00,1.0,1125.0,Apr,3247
3,6369,2021-04-17,t,$60.00,$60.00,1.0,1125.0,Apr,3247
4,6369,2021-04-18,t,$60.00,$60.00,1.0,1125.0,Apr,3247
...,...,...,...,...,...,...,...,...,...
7161251,49103093,2022-04-10,f,$28.00,$35.00,1.0,1125.0,Apr,3247
7161252,49103093,2022-04-11,f,$28.00,$35.00,1.0,1125.0,Apr,3247
7161253,49103093,2022-04-12,f,$28.00,$35.00,1.0,1125.0,Apr,3247
7161254,49103093,2022-04-13,f,$28.00,$35.00,1.0,1125.0,Apr,3247


In [7]:
df['Month'].unique()

array(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',
       'Jan', 'Feb', 'Mar'], dtype=object)

# Create Spark & Delta Lake

In [8]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e723294-8554-4a57-ae4b-0cbc5755cbb6;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.2.0/delta-spark_2.12-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.2.0!delta-spark_2.12.jar (1118ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.2.0/delta-storage-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.2.0!delta-storage.jar (250ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (251ms)
:: resolution report :: resolve 4120ms :: artifacts dl

In [9]:
HR = spark.createDataFrame(df)

In [10]:
HR.show()

24/05/28 09:13:45 WARN TaskSetManager: Stage 0 contains a task of very large size (92418 KiB). The maximum recommended task size is 1000 KiB.
24/05/28 09:13:51 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+----------+----------+---------+------+--------------+--------------+--------------+-----+-----------------+
|listing_id|      date|available| price|adjusted_price|minimum_nights|maximum_nights|Month|Subway_Passengers|
+----------+----------+---------+------+--------------+--------------+--------------+-----+-----------------+
|    167183|2021-04-15|        f|$45.00|        $45.00|           1.0|           5.0|  Apr|             3247|
|      6369|2021-04-15|        t|$60.00|        $60.00|           1.0|        1125.0|  Apr|             3247|
|      6369|2021-04-16|        t|$60.00|        $60.00|           1.0|        1125.0|  Apr|             3247|
|      6369|2021-04-17|        t|$60.00|        $60.00|           1.0|        1125.0|  Apr|             3247|
|      6369|2021-04-18|        t|$60.00|        $60.00|           1.0|        1125.0|  Apr|             3247|
|      6369|2021-04-19|        t|$60.00|        $60.00|           1.0|        1125.0|  Apr|             3247|
|      636